## Problem 1-4 : NorthWinddatset https://relational.fit.cvut.cz/dataset/Northwind

How to download the dataset

To download the dataset you need to create connection in your mysql workbench

The datasets are publicly available directly from MariaDB database.

* Open your favourite MariaDB client (MySQL Workbench)
Use following credentials:
```
    hostname: relational.fit.cvut.cz
    port: 3306
    username: guest
    password: relational
```
Drive link: https://drive.google.com/drive/folders/1FMi6g1KDHbn17ikJSi7bE9-THywlxhdu?usp=sharing

### `Q-1`: Rank Employee in terms of revenue generation. Show employee id, first name, revenue, and rank


### `Q-2`: Show All products cumulative sum of units sold each month.


### `Q-3`: Show Percentage of total revenue by each suppliers



### `Q-4:` Show Percentage of total orders by each suppliers



### `Q-5`:Show All Products Year Wise report of totalQuantity sold, percentage change from last year.





```SQL
-- Q1

SELECT e.EmployeeID, e.FirstName, SUM(od.UnitPrice * od.Quantity) AS revenue,
RANK() OVER(ORDER BY SUM(od.UnitPrice * od.Quantity) DESC) AS EmpRank
FROM nw_orders o
JOIN nw_order_details od
ON od.OrderID = o.OrderID
JOIN nw_employees e
ON e.EmployeeID = o.EmployeeID
GROUP BY e.EmployeeID, e.FirstName
ORDER BY EmpRank;

-- Q2

SELECT p.ProductID, MONTH(o.OrderDate) AS 'Month',
SUM(od.Quantity) AS 'QuantitySum',
SUM(SUM(od.Quantity)) OVER (PARTITION BY p.ProductID ORDER BY MONTH(o.OrderDate) ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS QuantityCumSum
FROM nw_products p
JOIN nw_order_details od
ON p.ProductID = od.ProductID
JOIN nw_orders o
ON o.OrderID = o.OrderID
GROUP BY p.ProductID, MONTH(o.OrderDate);

-- Q3

SELECT s.SupplierID,
SUM(od.UnitPrice * od.Quantity) AS Revenue,
SUM(od.UnitPrice * od.Quantity)/SUM(SUM(od.UnitPrice * od.Quantity)) OVER() * 100 AS PercentTotalRevenue
FROM nw_suppliers s
JOIN nw_products p
ON s.SupplierID = p.SupplierID
JOIN nw_order_details od
ON od.ProductID = p.ProductID
GROUP BY s.SupplierID
ORDER BY Revenue DESC;

-- Q4

SELECT s.SupplierID,
COUNT(DISTINCT od.OrderID) AS 'NumberOfOrder',
COUNT(DISTINCT od.OrderID) / SUM(COUNT(DISTINCT od.OrderID)) OVER() * 100 AS 'PercentTotalOrder'
FROM nw_suppliers s JOIN nw_products p ON s.SupplierID = p.SupplierID
JOIN nw_order_details od ON od.ProductID = p.ProductID
GROUP BY s.SupplierID
ORDER BY NumberOfOrder;

-- Q5
SELECT *,
100 * (Quantity - LAG(Quantity) OVER(PARTITION BY ProductID ORDER BY ProductID, Year)) /
LAG(Quantity) OVER(PARTITION BY ProductID ORDER BY ProductID, Year) PercentageChange
FROM (SELECT p.ProductID, YEAR(o.OrderDate) Year, SUM(od. Quantity) AS 'Quantity'
FROM nw_orders o JOIN nw_order_details od ON od.OrderID = o.OrderID
JOIN nw_products p ON p.ProductID = od.ProductID
GROUP BY p.ProductID, YEAR(o.OrderDate)
ORDER BY p.ProductID, YEAR(o.OrderDate)) t;
```

## Problem 6 - 15

For the next problems, you can get the dataset as well as the details [here](https://www.kaggle.com/datasets/thedevastator/drug-performance-evaluation?select=Drug_clean.csv) (Use the *cleaned* csv file).

### **`Problem-6:`** For each condition, what is the average satisfaction level of drugs that are "On Label" vs "Off Label"?


### **`Problem-7:`** For each drug type (RX, OTC, RX/OTC), what is the average ease of use and satisfaction level of drugs with a price above the median for their type?


### **`Problem 8:`** What is the cumulative distribution of EaseOfUse ratings for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and cumulative distribution. (Use the built-in method and the manual method by calculating on your own. For the manual method, use the "ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW" and see if you get the same results as the built-in method.)


### **`Problem 9:`** What is the median satisfaction level for each medical condition? Show the results in descending order by median satisfaction level. (Don't repeat the same rows of your result.)



### **`Problem 10:`** What is the running average of the price of drugs for each medical condition? Show the results in ascending order by medical condition and drug name.



### **`Problem 11:`** What is the percentage change in the number of reviews for each drug between the previous row and the current row? Show the results in descending order by percentage change.



### **`Problem 12:`** What is the percentage of total satisfaction level for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and percentage of total satisfaction.



### **`Problem 13:`** What is the cumulative sum of effective ratings for each medical condition and drug form combination? Show the results in ascending order by medical condition, drug form and the name of the drug.



### **`Problem-14:`** What is the rank of the average ease of use for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by rank and drug type.



### **`Problem-15:`** For each condition, what is the average effectiveness of the top 3 most reviewed drugs?

```SQL
-- SELECT * FROM task_37.drug;
SELECT * FROM drug_clean;

-- Q6
WITH temp_df AS 
(SELECT `Condition`, Indication, Satisfaction,
ROUND(AVG(Satisfaction) OVER(PARTITION BY `Condition`, Indication 
								  ORDER BY Satisfaction 
                                  ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING), 2) AS avg_satisfaction,
DENSE_RANK() OVER(PARTITION BY `Condition`, Indication ORDER BY Satisfaction) AS rank_num
FROM drug_clean)

SELECT `Condition`, Indication, avg_satisfaction
FROM temp_df
WHERE rank_num = 1;

-- Q7
WITH temp_df as (
    SELECT Type,
        AVG(EaseOfUse) OVER(PARTITION BY Type) AS avg_ease_of_use,
        AVG(Satisfaction) OVER(PARTITION BY Type) AS avg_satisfaction
    FROM (
        SELECT
            Type, Price,
            PERCENTILE_CONT(0.5) WITHIN GROUP (
                ORDER BY Price
            ) OVER (PARTITION BY Type) AS median_price,
            EaseOfUse,
            Satisfaction
        FROM drugs WHERE Type IN ('RX', 'OTC', 'RX/OTC')
    ) AS subquery
    WHERE Price >= median_price
)

SELECT Type, avg_ease_of_use, avg_satisfaction FROM temp_df GROUP BY Type;

-- Q8
SELECT Type, EaseOfUse,
	   COUNT(*) OVER(PARTITION BY Type ORDER BY EaseOfUse) * 1.0 / 
	   COUNT(*) OVER(PARTITION BY Type) AS cumulative_dist_manual,
CUME_DIST() OVER(PARTITION BY Type ORDER BY EaseOfUse) AS `cumulative_dist_builtin`
FROM drug_clean
WHERE Type IN ('RX', 'OTC', 'RX/OTC')
ORDER BY Type, cumulative_dist_builtin DESC
;

-- Q9
WITH temp_df AS (
    SELECT drugs.Condition,
       PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY drugs.Satisfaction) 
       OVER (PARTITION BY drugs.Condition) AS median_satisfaction
    FROM drugs
)

SELECT temp_df.Condition, temp_df.median_satisfaction
FROM temp_df
GROUP BY temp_df.Condition
ORDER BY temp_df.median_satisfaction DESC;


-- Q10
SELECT `Condition`, 
       Drug, 
       ROUND(Price, 2) AS rounded_price,
       ROUND(AVG(Price) OVER (PARTITION BY `Condition` 
                              ORDER BY Drug 
                              ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 2) AS running_avg_price
FROM drug_clean
ORDER BY `Condition`, Drug;


-- Q11
SELECT `Condition`, Drug, Reviews,
((Reviews - LAG(Reviews) OVER(PARTITION BY `Condition`, Drug ORDER BY Reviews DESC)) * 100 /
LAG(Reviews) OVER(PARTITION BY `Condition`, Drug ORDER BY Reviews DESC)) AS pct_change
FROM drug_clean
ORDER BY pct_change DESC;

-- Q12
WITH temp_df AS (
	SELECT Type, Satisfaction,
    ROUND(SUM(Satisfaction) OVER (PARTITION BY Type) * 100.0 /
		  SUM(Satisfaction) OVER(), 2) AS pct_total_satisfaction
	FROM drug_clean
    WHERE Type IN ('RX', 'OTC', 'RX/OTC')
    ORDER BY Type, pct_total_satisfaction DESC
)

SELECT Type, pct_total_satisfaction
FROM temp_df
GROUP BY Type, pct_total_satisfaction;

-- Q13
WITH temp_df AS 
(SELECT `Condition`, Form, Drug, Effective,
SUM(Effective) OVER(PARTITION BY `Condition`, Form ORDER BY Drug 
					ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum
FROM drug_clean)

SELECT *
FROM temp_df
ORDER BY `Condition`, Form, Drug;

-- Q14
SELECT Type,
AVG(EaseOfUse) AS 'avg_ease_of_use',
RANK() OVER(ORDER BY AVG(EaseOfUse) DESC) AS 'rank'
FROM drug_clean
WHERE Type IN ('RX', 'OTC', 'RX/OTC')
GROUP BY Type;

SELECT
  Type,
  AVG(EaseOfUse) AS average_ease_of_use,
  RANK() OVER (ORDER BY AVG(EaseOfUse) DESC) AS 'rank'
FROM drug_clean
WHERE Type IN ('RX', 'OTC', 'RX/OTC')
GROUP BY Type;

-- Q15
SELECT * FROM (
SELECT 
		`Condition`, Drug,
		ROUND(Reviews, 2) AS 'Reviews',
        ROUND(AVG(Effective) OVER(PARTITION BY `Condition`, Drug 
								  ORDER BY Reviews DESC
                                  ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING), 2) AS avg_effectiveness,
		RANK() OVER (PARTITION BY `Condition`
					 ORDER BY Reviews DESC) AS rank_num
		FROM drug_clean) t
WHERE t.rank_num <= 3;

SELECT * FROM (
SELECT  `Condition`, Drug,
        ROUND(Reviews, 2) AS 'Reviews',
        ROUND(AVG(Effective) OVER (PARTITION BY `Condition`, Drug
								   ORDER BY Reviews DESC
								   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING), 2) AS avg_effectiveness,
		RANK() OVER (PARTITION BY `Condition`
					 ORDER BY Reviews DESC) AS rank_num
		FROM drug_clean) t
WHERE rank_num <= 3;
```